<a href="https://www.kaggle.com/code/pogscafe/fooocus?scriptVersionId=150876834" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Installation

In [ ]:
HOME_FOLDER = '/kaggle/working'
TEMP_FOLDER = '/kaggle/temp'
update = False

checkpoints = f'{HOME_FOLDER}/Fooocus/models/checkpoints'
temp_models = f'{HOME_FOLDER}/Fooocus/models/checkpoints/temp_models'

!mkdir $TEMP_FOLDER

import os
from os import path

%cd $HOME_FOLDER

if not path.exists('Fooocus'):
    get_ipython().system('git clone https://github.com/lllyasviel/Fooocus.git')
    #get_ipython().system('git clone --depth 1 --branch 1.0.35 https://github.com/lllyasviel/Fooocus.git')
else:
    get_ipython().system(f'find {HOME_FOLDER}/Fooocus/models/checkpoints -maxdepth 1 -type l -delete') # delete any symlinks to temp models from a previous run
    
%cd Fooocus

if update:
    get_ipython().system('git pull')
    
!pip install -r requirements_versions.txt 
!pip install torch torchvision --force-reinstall --index-url https://download.pytorch.org/whl/cu117
!rm -rf /opt/conda/lib/python3.10/site-packages/pytz-2023.3.dist-info
!rm -rf /opt/conda/lib/python3.10/site-packages/PyYAML-6.0.1.dist-info
!pip install pyyaml pytz 

!mamba install openssh -y

# Start the WebUI

**Starting the Web UI with ngrok**  
* Make sure to put your ngrok token in the Ngrok_token variable. The token can be obtained from https://ngrok.com
* If you have a static domain, put your ngrok domain in the Ngrok_domain variable.
* Wait for the line that says "App started successful. Use the app with http://127.0.0.1:7865/ or 127.0.0.1:7865" 
    * If the port is no longer 7865 in the latest version, please change the port variable as well
* Visit your ngrok URL (either your static domain, or the ngrok url displayed in the output)

In [ ]:
# Starting the Web UI with ngrok

# --- Variables ---

Ngrok_token = "" #@param {type:"string"}
# Put your ngrok token here (obtainable from https://ngrok.com)
# Example: Ngrok_token = "2Fw13n4GcJT12g7mSDUC62cdNGb_5svdjf3Gg5vfhr4nGr5gF"

Ngrok_domain = "" # optional, leave empty if you don't have a domain

port = 7865

# -----------------


!pip install pyngrok

from pyngrok import ngrok, conf
import fileinput
import sys
import gc

gc.collect()

if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(port , pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token),
                    bind_tls=True, domain=Ngrok_domain).public_url
  print(srv)
  get_ipython().system(f"python {HOME_FOLDER}/Fooocus/entry_with_update.py ")
else:
  print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the ngrok_token field.')

***   
**Option 2: Starting the Web UI with RemoteMoe**  
* Wait for the line that says "App started successful. Use the app with http://127.0.0.1:7865/ or 127.0.0.1:7865"   
* Click the link that ends with .remote.moe

In [ ]:
#Option 2: Starting the Web UI with RemoteMoe
port = 7865

!mkdir  ~/.ssh/
!touch  ~/.ssh/known_hosts
!ssh-keyscan -t rsa remote.moe >> ~/.ssh/known_hosts
!rm /root/.ssh/id_rsa
!ssh-keygen -t rsa -b 4096 -f /root/.ssh/id_rsa -q -N ""
!python $HOME_FOLDER/Fooocus/entry_with_update.py & ssh -R 80:127.0.0.1:$port -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe 

# Install Models

In [ ]:
# Install a model to permanent storage
# Make sure Persistence is set to "Files only" or "Variables and Files"
model_url = 'https://civitai.com/api/download/models/169740?type=Model&format=SafeTensor&size=full&fp=fp16'
model_name = 'ZavyChroma.safetensors'

%cd $checkpoints
#!wget -O 'model.safetensors' 'https://civitai.com/api/download/models/198530?type=Model&format=SafeTensor&size=full&fp=fp16'
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

In [ ]:
# Install a LoRA to permanent storage
model_url = 'https://civitai.com/api/download/models/137124?type=Model&format=SafeTensor'
model_name = 'DreamArt.safetensors'

LORA_FOLDER = HOME_FOLDER + '/Fooocus/models/loras'
%cd $LORA_FOLDER
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

In [ ]:
# Install a model to temporary storage
model_url = 'https://civitai.com/api/download/models/160191?type=Model&format=SafeTensor&size=full&fp=fp16'
model_name = 'YamersRealism.safetensors'

%cd $TEMP_FOLDER
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

if not path.exists(f'{HOME_FOLDER}/Fooocus/models/checkpoints/{model_name}'):
    get_ipython().system(f'ln -s {TEMP_FOLDER}/{model_name} {HOME_FOLDER}/Fooocus/models/checkpoints/')